In [1]:
import os
import h5py 
import pickle
import numpy as np

from skimage import *

##  Define the Training paths.

In [2]:
training_path_1 = '/home/wentao/project/keras_training/'
training_path_2 = '/home/wentao/UNet-MRI-Reconstruction/keras_training/'
training_path_3 = '/home/wentao/extra_space/keras_training/'

In [3]:
training_paths = [training_path_1, training_path_2, training_path_3]

In [4]:
def get_subdirectories(path):
    return [(name, os.path.join(path, name)) for name in os.listdir(path)
            if os.path.isdir(os.path.join(path, name))]

In [5]:
def calculate_metrics(result_file):
    
    f = h5py.File(training_result,'r') 

    result = np.array(f.get('result'))
    truth = np.array(f.get('truth'))
    imag = np.array(f.get('input'))
    
    psnr = []
    mse = []
    nrmse = []
    ssim = []
    for i in range(0, truth.shape[0]):
        psnr.append(measure.compare_psnr(result[i],truth[i], 1))
        mse.append(measure.compare_mse(result[i], truth[i]))
        nrmse.append(measure.compare_nrmse(result[i], truth[i]))
        ssim.append(measure.compare_ssim(result[i], truth[i], data_range=1))
    
    # find the best and the worst images by ssim value  
    best_image_index = ssim.index(max(ssim))
    worst_image_index = ssim.index(min(ssim))
    
    print("mse: {0:.4f}, nrmse: {1:.4f}, psnr: {2:.4f}, ssim: {3:.4f} ".format(np.mean(mse), np.mean(nrmse), np.mean(psnr), np.mean(ssim)))
    print("best_ssim: {0:.4f}, worst_ssim: {1:.4f}".format(ssim[best_image_index], ssim[worst_image_index]))
    return [np.mean(mse), np.mean(nrmse), np.mean(psnr), np.mean(ssim), ssim[best_image_index], ssim[worst_image_index]]

In [6]:
result = []

##  Look for h5 file in the subdirectories recursively

In [7]:
for training_path in training_paths:
    print(training_path)
    for task_name, path in get_subdirectories(training_path):
        print(task_name)
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in [f for f in filenames if f.endswith(".h5")]:
                training_result = os.path.join(dirpath, filename)
                metric = calculate_metrics(training_result)
                result.append((task_name, metric))
                print()

/home/wentao/project/keras_training/
UNet2D2D_SSIM_loss_no_regularization
mse: 0.0020, nrmse: 0.1756, psnr: 27.6945, ssim: 0.8776 
best_ssim: 0.9523, worst_ssim: 0.7699

mse: 0.0024, nrmse: 0.1886, psnr: 26.6277, ssim: 0.8478 
best_ssim: 0.9441, worst_ssim: 0.6771

UNet3D_old_MAE_loss_no_regularization
mse: 0.0022, nrmse: 0.1893, psnr: 27.1953, ssim: 0.8392 
best_ssim: 0.9375, worst_ssim: 0.7034

UNet2D2D_PSNR_loss_no_regularization
mse: 0.0029, nrmse: 0.2375, psnr: 26.7484, ssim: 0.8324 
best_ssim: 0.9429, worst_ssim: 0.6334

mse: 0.0027, nrmse: 0.2227, psnr: 26.8118, ssim: 0.8069 
best_ssim: 0.9266, worst_ssim: 0.6123

UNet3D_old_MAE_loss_l1_regularization
UNet2D2D_MAE_loss_no_regularization
mse: 0.0024, nrmse: 0.2074, psnr: 26.9778, ssim: 0.8288 
best_ssim: 0.9296, worst_ssim: 0.7131

UNet3D_old_MSE_loss_l2_regularization
mse: 0.0030, nrmse: 0.2223, psnr: 25.8929, ssim: 0.7805 
best_ssim: 0.9036, worst_ssim: 0.6013

mse: 0.0029, nrmse: 0.2156, psnr: 25.9704, ssim: 0.7944 
best_ssim:

mse: 0.0059, nrmse: 0.3076, psnr: 22.5040, ssim: 0.6006 
best_ssim: 0.8026, worst_ssim: 0.3577

mse: 0.0066, nrmse: 0.3434, psnr: 22.0814, ssim: 0.5541 
best_ssim: 0.7576, worst_ssim: 0.3475

UNet2D1D_PSNR_loss_instance_norm
mse: 0.0058, nrmse: 0.3025, psnr: 22.6132, ssim: 0.5696 
best_ssim: 0.7179, worst_ssim: 0.3712

UNet3D_MAE_loss_dropout_0.2
mse: 0.0059, nrmse: 0.3199, psnr: 22.5790, ssim: 0.5777 
best_ssim: 0.7963, worst_ssim: 0.3448

UNet2D1D_MSE_loss_dropout_0.2
mse: 0.0060, nrmse: 0.3133, psnr: 22.4332, ssim: 0.5920 
best_ssim: 0.7952, worst_ssim: 0.3566

UNet3D_MSE_loss_dropblock_0.9_5
mse: 0.0056, nrmse: 0.3054, psnr: 22.7742, ssim: 0.5935 
best_ssim: 0.7930, worst_ssim: 0.3642

UNet2D1D_SSIM_loss_dropblock_0.9_5
mse: 0.0059, nrmse: 0.2983, psnr: 22.4934, ssim: 0.6423 
best_ssim: 0.8233, worst_ssim: 0.3993

UNet2D2D_MAE_loss_l2_regularization
mse: 0.0026, nrmse: 0.2131, psnr: 26.7312, ssim: 0.8311 
best_ssim: 0.9442, worst_ssim: 0.6631

UNet3D_MSE_loss_l2_regularization
mse:

## Sort the result by SSIM value.

In [8]:
result.sort(key=lambda r:r[1][3], reverse=True)

In [9]:
result

[('UNet3D_old_SSIM_loss_no_regularization',
  [0.002016376608234875,
   0.17676645659845291,
   27.639025245804746,
   0.8782221176465951,
   0.9554262102372293,
   0.7767683276251596]),
 ('UNet2D2D_SSIM_loss_no_regularization',
  [0.00199017724690729,
   0.17560795468353496,
   27.69450816257319,
   0.877606626530342,
   0.952268952715654,
   0.7698848257240287]),
 ('UNet3D_old_SSIM_loss_no_regularization',
  [0.001998331481373661,
   0.17300310229943905,
   27.6411300955369,
   0.8766105939778008,
   0.9570466222540702,
   0.7547330556147073]),
 ('UNet3D_SSIM_loss_l2_regularization',
  [0.0020523418988982714,
   0.18244696844617242,
   27.5434170489525,
   0.8759947718422536,
   0.9550018470830206,
   0.7605980254637473]),
 ('UNet3D_old_PSNR_loss_no_regularization',
  [0.0017199695243932782,
   0.16032955454795889,
   28.301902277248136,
   0.8609208938858088,
   0.954010071853662,
   0.7489076544952885]),
 ('UNet2D2D_SSIM_loss_l2_regularization',
  [0.002493312762632006,
   0.205842

In [10]:
with open('desktop_result.pkl', 'wb') as f:
    pickle.dump(result, f)